In this notebook we are going to focus on features which will be derived from time domain characteristics.

In [1]:
import numpy as np
import math
from scipy.signal import argrelextrema, find_peaks

In [2]:
path = "/Users/ecem/Desktop/phonocardiogram/data/PV/present"

In [3]:
waves = np.load(path + "/present_PV_waves_10sec.npy", allow_pickle= True)

In [4]:
waves.shape

(1198, 40000)

## Energy:

In [5]:
def energy(data):
    return (1 / len(data)) * np.sum(np.square(data))

In [6]:
energy_ = []
for i in range(waves.shape[0]):
    energy_.append(energy(waves[i]))

In [7]:
energy_ = np.array(energy_)

In [8]:
energy_.shape

(1198,)

## Entropy:

Entropy is a thermodynamics concept that measures the molecular disorder in a closed system. This concept is used in nonlinear dynamical systems to quantify the degree of complexity. Entropy is an interesting tool for analyzing time series, as it does not consider any constraints on the probability distribution [7]. Shannon entropy (ShEn) and conditional entropy (ConEn) are the basic measures used for evaluating entropy. ShEn and ConEn measure the amount of information and the rate of information generation, respectively [1]. Based on these measures, other entropy measures have been introduced for evaluating the complexity of time series

In [9]:
def entropy(data, num_short_blocks=10):
    eol = np.sum(np.square(data))
    win_len = len(data)
    sub_win_len = math.floor(win_len / num_short_blocks)

    if win_len != sub_win_len * num_short_blocks:
        data = data[0:sub_win_len * num_short_blocks]
    sub_wins = data.reshape(sub_win_len, num_short_blocks, order='F').copy()
    norm_sub_frame_energies = np.zeros((1, sub_wins.shape[1]))
    for i in range(sub_wins.shape[1]):
        norm_sub_frame_energies[0, i] = np.sum(np.square(sub_wins[:, i])) / (eol + np.spacing(1))
    energy_entropy = 0
    for i in range(norm_sub_frame_energies.shape[1]):
        energy_entropy -= norm_sub_frame_energies[0, i] * math.log(norm_sub_frame_energies[0, i] + np.spacing(1), 2)
    return energy_entropy

In [10]:
entropy(waves[0], num_short_blocks = 10)

3.1983124318121967

In [11]:
entropy_ = []
for i in range(waves.shape[0]):
    entropy_.append(entropy(waves[i], num_short_blocks = 10))

In [12]:
entropy_ = np.array(entropy_)
entropy_.shape

(1198,)

## discrete fourier transform

In [13]:
def dft(data, f_s = 4000, p=0):
    win_len = len(data)
    fft = np.abs(np.fft.fft(data)) / win_len
    if not p:
        fft = fft[0:math.ceil(win_len)]
        f_req = (f_s / 2) * np.arange(0, np.ceil(win_len / 2) + 1) / np.ceil(win_len / 2)
    else:
        fft = np.fft.fftshift(fft)
        if win_len % 2:
            f_req = np.arange(-(win_len - 1) / 2, (win_len - 1) / 2 + 1)
        else:
            f_req = np.arange(-win_len / 2, win_len / 2)
    fft_1 = np.abs(fft)/win_len
    fft_2 = fft_1[1:(round(win_len / 2) + 1)]
    fft_2 = 2*fft_2
    return fft_2, f_req

## Spectral Entropy

In [14]:
spec_ent = []
for i in range(waves.shape[0]):
    fft, _ = dft(waves[i])
    spec_ent.append(entropy(fft, 10))

In [15]:
spec_ent = np.array(spec_ent)

In [16]:
spec_ent.shape

(1198,)

## Spectral RollOff

Spectral rolloff is the frequency below which a specified percentage of the total spectral energy, e.g. 85%, lies

In [17]:
def spectral_rolloff(data, c=0.90):
    total_energy = np.sum(np.square(data))
    curr_energy = 0
    count_fft = 0
    fft_len = len(data)
    while curr_energy <= c * total_energy and count_fft <= fft_len:
        curr_energy += data[count_fft] ** 2
        count_fft += 1
    count_fft -= 1
    return (count_fft - 1) / fft_len

In [18]:
fft, _ = dft(waves[0])
spectral_rolloff(fft)

0.04955

In [19]:
rolloff = []
for i in range(waves.shape[0]):
    fft, _ = dft(waves[i])
    rolloff.append(spectral_rolloff(fft))

In [20]:
rolloff = np.array(rolloff)
rolloff.shape

(1198,)

## Spectral Centroid

The spectral centroid (Wikipedia) indicates at which frequency the energy of a spectrum is centered upon. 

## SOR: burada bir pozitif negatif sıkıntısı vardı. Aşağıdaki gibi çözdüm sıkıntı oluyor mu ? 

In [21]:
def spectral_centroid(data, f_s = 4000):
    fft_len = len(data)
    m = np.transpose((f_s / (2 * fft_len)) * np.arange(1, fft_len+1))
    data = data / np.max(data)
    c = np.sum(np.multiply(m, data)) / (np.sum(data) + np.spacing(1))
    k = np.sum(np.square(m - c) * data)
    l = (np.sum(data) + np.spacing(1))
    
    if k*l <0 :
        s = math.sqrt( -1* k / l ) / (f_s / 2)
        c = c / (f_s / 2)
        return c
    else:
        s = math.sqrt( k / l ) / (f_s / 2)
        c = c / (f_s / 2)
        return c

In [22]:
centroid = []
for i in range(waves.shape[0]):
    fft, _ = dft(waves[i])
    centroid.append(spectral_centroid(fft))

In [23]:
centroid = np.array(centroid)
centroid.shape

(1198,)

## Spectral spread:

In [24]:
def spectral_spread(data, f_s=4000):
    fft_len = len(data)
    m = np.transpose((f_s / (2 * fft_len)) * np.arange(1, fft_len+1))
    data = data / np.max(data)
    c = np.sum(np.multiply(m, data)) / (np.sum(data) + np.spacing(1))
    k = np.sum(np.square(m - c) * data)
    l = (np.sum(data) + np.spacing(1))
    
    if k*l <0 :
        s = math.sqrt( -1* k / l ) / (f_s / 2)
        c = c / (f_s / 2)
        return c
    else:
        s = math.sqrt( k / l ) / (f_s / 2)
        c = c / (f_s / 2)
        return s

In [25]:
spread = []
for i in range(waves.shape[0]):
    fft, _ = dft(waves[i])
    spread.append(spectral_spread(fft))

In [26]:
spread = np.array( spread)
spread.shape

(1198,)

# Now lets create their dataframes

In [27]:
import pandas as pd

RuntimeError: module compiled against API version 0xe but this version of numpy is 0xd

In [28]:
energy_df = pd.DataFrame(energy_, columns =["energy"])
energy_df.head()

,energy
0,0.001070
1,0.001786
2,0.001811
3,0.001241
4,0.000854


In [29]:
entropy_df = pd.DataFrame(entropy_, columns =["entropy"])
entropy_df.head()

,entropy
0,3.198312
1,3.170767
2,2.880738
3,3.223023
4,3.105578


In [30]:
spec_ent_df = pd.DataFrame(spec_ent, columns = ["spectral entropy"])
spec_ent_df.head()

,spectral entropy
0,0.045529
1,0.053781
2,0.013223
3,0.027395
4,0.131291


In [31]:
rolloff_df = pd.DataFrame(rolloff, columns =["spectral rolloff"])
rolloff_df.head()

,spectral rolloff
0,0.04955
1,0.04745
2,0.04500
3,0.04370
4,0.05265


In [32]:
centroid_df = pd.DataFrame(centroid, columns =["spectral centorid"])
centroid_df.head()

,spectral centorid
0,0.041271
1,0.042624
2,0.033304
3,0.051339
4,0.049568


In [33]:
spread_df = pd.DataFrame(spread, columns =["spectral spread"])
spread_df.head()

,spectral spread
0,0.063945
1,0.067752
2,0.044521
3,0.105631
4,0.075318


## Now concatenate all features

In [34]:
df = pd.concat([energy_df, entropy_df, spec_ent_df, rolloff_df, centroid_df, spread_df], axis =1)

In [35]:
df

,energy,entropy,spectral entropy,spectral rolloff,spectral centorid,spectral spread
0,0.001070,3.198312,0.045529,0.04955,0.041271,0.063945
1,0.001786,3.170767,0.053781,0.04745,0.042624,0.067752
2,0.001811,2.880738,0.013223,0.04500,0.033304,0.044521
3,0.001241,3.223023,0.027395,0.04370,0.051339,0.105631
4,0.000854,3.105578,0.131291,0.05265,0.049568,0.075318
...,...,...,...,...,...,...
1193,0.002508,2.921813,0.139406,0.03340,0.045504,0.056661
1194,0.001985,2.761107,0.084441,0.01670,0.041503,0.068024
1195,0.001082,2.874245,0.325370,0.07365,0.056984,0.072238
1196,0.001607,2.882222,0.251177,0.04575,0.065567,0.113860


In [36]:
df.to_csv("/Users/ecem/Desktop/phonocardiogram/data/PV/present/PV-time-domain-features.csv" )